In [1]:
# Por si no lo montáis desde el menú lateral
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### PRIMER DÍA. FICHERO CON COORDENADAS Y HORAS DE INSTALACIÓN NECESARIAS. A LAS DIRECCIONES QUE NO LOCALIZA SE ASIGNAN LAS COORDENADAS DEL CÓDIGO POSTAL

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Crear objeto geolocalizador
geolocator = Nominatim(user_agent="mi_aplicacion", timeout=10)

# Archivo día 1
df = pd.read_excel('/content/drive/MyDrive/Dia1_20250227.xlsx', sheet_name='Hoja1', engine='openpyxl')

# Incluimos una pausa para facilitar la búsqueda y que no de errores
def obtener_coordenadas_con_pausa(row):
    direccion = f"{row['CALLE']}, {row['NUMERO']}, {row['CP']}, Madrid, España"
    try:
        location = geolocator.geocode(direccion)
        time.sleep(1)  # Pausa de 1 segundo entre solicitudes
        if location:
            return location.latitude, location.longitude, False
        else:
            # Si no puede localizar las coordenadas por dirección asigna la del CP
            direccion_cp = f"{row['CP']}, Madrid, España"
            location_cp = geolocator.geocode(direccion_cp)
            time.sleep(1)  # Pausa de 1 segundo entre solicitudes
            if location_cp:
                return location_cp.latitude, location_cp.longitude, True
            else:
                return None, None, False
    except GeocoderTimedOut:
        return obtener_coordenadas_con_pausa(row)

# Se aplica la función a cada fila del fichero
df['Latitud'], df['Longitud'], df['Solo_CP'] = zip(*df.apply(obtener_coordenadas_con_pausa, axis=1))

# Se añade la columna "Horas_instalacion" asignando las horas en función de la familia del producto
def calcular_horas_instalacion(row):
    if row['FAMILIA'] == 810:
        return row['UNIDADES'] * 2
    elif row['FAMILIA'] == 811:
        return row['UNIDADES'] * 4
    elif row['FAMILIA'] == 812:
        return row['UNIDADES'] * 6
    elif row['FAMILIA'] == 813:
        return row['UNIDADES'] * 6
    elif row['FAMILIA'] == 816:
        return row['UNIDADES'] * 2
    elif row['FAMILIA'] == 818:
        return row['UNIDADES'] * 2
    elif row['FAMILIA'] == 820:
        return row['UNIDADES'] * 4
    elif row['FAMILIA'] == 822:
        return row['UNIDADES'] * 8
    elif row['FAMILIA'] == 824:
        return row['UNIDADES'] * 6
    else:
        return None
df['Horas_instalacion'] = df.apply(calcular_horas_instalacion, axis=1)

# Guardar el fichero
output_path = '/content/drive/MyDrive/Dia1_20250227_coord_horas.xlsx'
df.to_excel(output_path, index=False)

# Cargar el archivo Excel guardado para aplicar formato de color rojo a las coordenadas asignadas solo por CP
wb = load_workbook(output_path)
ws = wb.active

# Definir el estilo de color rojo para las celdas
red_fill = PatternFill(start_color="FFFF0000", end_color="FFFF0000", fill_type="solid")

# Aplicar el estilo a las celdas correspondientes en las columnas de Latitud y Longitud
for index, row in df.iterrows():
    if row['Solo_CP']:
        ws[f'W{index + 2}'].fill = red_fill  # Columna C para Latitud
        ws[f'X{index + 2}'].fill = red_fill  # Columna D para Longitud

# Guardar los cambios en el archivo Excel
wb.save(output_path)


Las coordenadas y la nueva columna 'Horas_instalacion' se han añadido y el archivo se ha guardado como 'Dia1_20250227_coord_horas.xlsx'. Las coordenadas asignadas solo por CP se han marcado en color rojo.


### AGRUPAR POR CERCANIA A LOS CLIENTES. EPS = 0.02 rádio de 2,22KM. Capacidad maxima para asignar 8h de instalación. Hay controles en en informe para poder revisar los cluster asignados: si el número de horas del cluster es 8h se revisa para verificar el número de domicilios afectados. Si un cluster supera las 8 horas, normalmente mismo domicilio, se indica en el informe y se gestiona manualmente. Habría que asignar un cluster distinto para que en el siguiente paso asigne un instalador o tratarlo manualmente poniendo "manual"

In [2]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

data = pd.read_excel('/content/drive/MyDrive/Dia1_20250227_coord_horas.xlsx', sheet_name='Sheet1')

# Función para agrupar por cercanía sin restricciones
def agrupar_por_cercania(data, eps=0.02, min_samples=1):
    agrupados = []

    # Iterar sobre cada FECHA COMP INICIAL
    for fecha in data["FECHA COMP INICIAL"].unique():
        subset = data[data["FECHA COMP INICIAL"] == fecha]

        # DBSCAN para agrupar por latitud y longitud
        coords = subset[["Latitud", "Longitud"]].values
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
        subset["cluster"] = db.labels_

        agrupados.append(subset)

    return pd.concat(agrupados)

# Se aplica la función y se guarda el resultado
agrupados = agrupar_por_cercania(data)
agrupados.to_excel("/content/drive/MyDrive/Clientes_agrupados_sin_restricciones.xlsx", index=False)

# Añadir columna "cluster_ajustado" según las reglas
def ajustar_clusters(data):
    data["cluster_ajustado"] = ""

    for cluster in data["cluster"].unique():
        cluster_data = data[data["cluster"] == cluster]
        total_hours = 0
        sub_cluster_id = 0
        lat_lon_dict = {}

        for index, row in cluster_data.iterrows():
            lat_lon = (row["Latitud"], row["Longitud"])

            if lat_lon in lat_lon_dict:
                # Se suman las horas de instalación para latitudes y longitudes iguales
                total_hours += row["Horas_instalacion"]
                data.loc[index, "cluster_ajustado"] = lat_lon_dict[lat_lon]
            else:
                # Verificar si hay una sola longitud y latitud en el cluster
                if len(cluster_data["Latitud"].unique()) == 1 and len(cluster_data["Longitud"].unique()) == 1:
                    threshold = 8
                else:
                    threshold = 6

                # Se suman las horas de instalación de todas las filas con la misma latitud y longitud
                same_lat_lon_hours = cluster_data[(cluster_data["Latitud"] == row["Latitud"]) & (cluster_data["Longitud"] == row["Longitud"])]["Horas_instalacion"].sum()

                if total_hours + same_lat_lon_hours > threshold:
                    sub_cluster_id += 1
                    total_hours = row["Horas_instalacion"]
                else:
                    total_hours += row["Horas_instalacion"]

                cluster_ajustado = f"{cluster}_{sub_cluster_id}"
                data.loc[index, "cluster_ajustado"] = cluster_ajustado
                lat_lon_dict[lat_lon] = cluster_ajustado

    # Se suma el total de "Horas_instalacion" de cada "cluster_ajustado" por "FECHA COMP INICIAL" y asignar "manual" si es mayor de 8
    for fecha in data["FECHA COMP INICIAL"].unique():
        fecha_data = data[data["FECHA COMP INICIAL"] == fecha]
        for cluster_ajustado in fecha_data["cluster_ajustado"].unique():
            if fecha_data[fecha_data["cluster_ajustado"] == cluster_ajustado]["Horas_instalacion"].sum() > 8:
                data.loc[(data["FECHA COMP INICIAL"] == fecha) & (data["cluster_ajustado"] == cluster_ajustado), "cluster_ajustado"] = "MANUAL"

    return data

# Ajustar los clusters y guardar el resultado
data_ajustada = ajustar_clusters(agrupados)

# Crear una columna con la suma de horas por cluster y fecha
data_ajustada['total_horas'] = data_ajustada.groupby(['FECHA COMP INICIAL', 'cluster_ajustado'])['Horas_instalacion'].transform('sum')

# Aplicar el estilo
def highlight_equal_8(s):
    is_equal_8 = s == 8
    return ['background-color: yellow' if v else '' for v in is_equal_8]

styled_data = data_ajustada.style.apply(highlight_equal_8, subset=['total_horas'])

# Guardar
styled_data.to_excel("/content/drive/MyDrive/Clientes_agrupados.xlsx", index=False)

print("Clusters ajustados y guardados en 'Clientes_agrupados.xlsx'")


<ipython-input-2-29d5121345d5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["cluster"] = db.labels_
<ipython-input-2-29d5121345d5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["cluster"] = db.labels_
<ipython-input-2-29d5121345d5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

Clusters ajustados y guardados en 'Clientes_agrupados.xlsx'


### ASIGNAR A LOS INSTALADORES POR CERCANIA

### ASIGNA A LOS INSTALADORES TENIENDO EN CUENTA SU CAPACIDAD DISPONIBLE Y ACTUALIZA EL FICHERO DE CAPACIDADES

In [13]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

agrupados = pd.read_excel("/content/drive/MyDrive/Clientes_agrupados.xlsx")
capacidad_instaladores_dia = pd.read_excel('/content/drive/MyDrive/Capacidad_instaladores_dia.xlsx', engine='openpyxl')

# Coordenadas de los clientes e instaladores
agrupados_coords = agrupados[["Latitud", "Longitud"]].values
instaladores_coords = capacidad_instaladores_dia[["Latitud", "Longitud"]].values
# Calcular la distancia euclidiana entre cada grupo y cada instalador
distancias = cdist(agrupados_coords, instaladores_coords, metric="euclidean")

# Se inicializa una columna para el instalador asignado
agrupados["Instalador Asignado"] = np.nan
agrupados["Latitud Instalador"] = np.nan
agrupados["Longitud Instalador"] = np.nan

# Diccionario para contar las asignaciones por instalador y fecha
asignaciones_por_instalador = {}

# Se itera sobre cada grupo
for i in range(len(agrupados)):
    fecha = agrupados.loc[i, "FECHA COMP INICIAL"]
    horas_instalacion = agrupados.loc[i, "Horas_instalacion"]

    # Se filtran los instaladores que tienen horas disponibles para la fecha
    instaladores_disponibles = capacidad_instaladores_dia[capacidad_instaladores_dia[fecha] >= horas_instalacion]

    # Se obtinen las coordenadas de los instaladores disponibles
    disponibles_coords = instaladores_disponibles[["Latitud", "Longitud"]].values

    # Calcular la distancia entre el grupo actual y los instaladores disponibles
    distancias_disponibles = cdist([agrupados_coords[i]], disponibles_coords, metric="euclidean")

    # Se asigna el instalador más cercano disponible
    while len(distancias_disponibles[0]) > 0:
        idx_min = distancias_disponibles.argmin()
        instalador_seleccionado = instaladores_disponibles.iloc[idx_min]
        horas_restantes = capacidad_instaladores_dia.loc[instalador_seleccionado.name, fecha] - horas_instalacion

        # Se verifica si el instalador ya tiene 2 asignaciones en la misma fecha
        if asignaciones_por_instalador.get((instalador_seleccionado["Instaladores"], fecha), 0) < 2:
            if horas_restantes >= 0:
                agrupados.loc[i, "Instalador Asignado"] = instalador_seleccionado["Instaladores"]
                agrupados.loc[i, "Latitud Instalador"] = instalador_seleccionado["Latitud"]
                agrupados.loc[i, "Longitud Instalador"] = instalador_seleccionado["Longitud"]

                # Restar las horas de instalación asignadas de las horas disponibles del instalador para esa fecha
                capacidad_instaladores_dia.loc[instalador_seleccionado.name, fecha] = horas_restantes

                # Actualizar el contador de asignaciones
                asignaciones_por_instalador[(instalador_seleccionado["Instaladores"], fecha)] = asignaciones_por_instalador.get((instalador_seleccionado["Instaladores"], fecha), 0) + 1
                break
        else:
            # Eliminar el instalador no válido y recalcular distancias
            distancias_disponibles = np.delete(distancias_disponibles, idx_min, axis=1)
            instaladores_disponibles = instaladores_disponibles.drop(instalador_seleccionado.name)

# Añadir la columna "Turno" al final del DataFrame
agrupados["Turno"] = np.nan

# Guardar el resultado en un nuevo archivo Excel
agrupados.to_excel("/content/drive/MyDrive/Agrupados_asignados_instaladores.xlsx", index=False)
capacidad_instaladores_dia.to_excel('/content/drive/MyDrive/Capacidad_instaladores_dia_actualizado.xlsx', index=False)


<ipython-input-13-5b3e2729851a>:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Prov24_inst1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  agrupados.loc[i, "Instalador Asignado"] = instalador_seleccionado["Instaladores"]


### SE ASIGNAN LOS TURNOS DE ENTREGA

In [ ]:
import pandas as pd

# Sobre el fichero anterior:
ruta_archivo = '/content/drive/MyDrive/Agrupados_asignados_instaladores.xlsx'
df = pd.read_excel(ruta_archivo, engine='openpyxl')

# Se define la función para asignar turnos
def asignar_turno(fila, turnos_asignados, turnos_instaladores, instaladores_cluster):
    cluster = fila['cluster_ajustado']
    fecha = fila['FECHA COMP INICIAL']
    horas = fila['Horas_instalacion']
    instalador = fila['Instalador Asignado']
    direccion = fila['DIRECCION']

    # Se verifica si el cluster y la fecha ya tienen turnos asignados
    if (cluster, fecha) in turnos_asignados:
        turnos_existentes = turnos_asignados[(cluster, fecha)]
    else:
        turnos_existentes = set()

    # Se verifica si el instalador ya tiene turnos asignados en la misma fecha
    if (instalador, fecha) in turnos_instaladores:
        clusters_instalador = turnos_instaladores[(instalador, fecha)]
    else:
        clusters_instalador = []

    # Se verifica si el cluster ya tiene instaladores asignados en la misma fecha
    if (cluster, fecha) in instaladores_cluster:
        lista_instaladores_cluster = instaladores_cluster[(cluster, fecha)]
    else:
        lista_instaladores_cluster = []

    # Criterios de asignación basado en las horas y los turnos
    if horas == 8:
        turno = 'DC'
    elif len(clusters_instalador) == 1:
        # Si el instalador solo tiene un cluster asignado en la misma fecha, asignar el turno contrario
        if clusters_instalador[0][1] == 'M':
            turno = 'T'
        else:
            turno = 'M'
    elif 'M' not in turnos_existentes and sum([h for h in turnos_asignados.get((cluster, fecha), []) if isinstance(h, int)]) < 7:
        turno = 'M'
    else:
        # Asignar turno de tarde 'T' solo si las horas no superan 4
        if horas <= 4:
            turno = 'T'
        else:
            turno = 'M'

    # Si hay dos o más instaladores en el mismo cluster y fecha con la misma dirección, asignar el mismo turno
    for inst in lista_instaladores_cluster:
        if inst[2] == direccion:
            turno = inst[1]
            break

    # Se verifica que un turno de tarde "T" no tenga más de 4 horas
    if turno == 'T' y horas > 4:
        turno = 'M'

    # Se verifica que un turno de mañana "M" no tenga más de 6 horas para el mismo instalador y día
    if turno == 'M' and sum([h for c, s in clusters_instalador if s == 'M' for h in turnos_asignados.get((c, fecha), []) if isinstance(h, int)]) + horas > 6:
        # Asignar turno de tarde 'T' solo si las horas no superan 4
        if horas <= 4:
            turno = 'T'
        else:
            turno = 'M'

    # Se actualizan los turnos
    turnos_asignados.setdefault((cluster, fecha), []).append(horas)
    turnos_asignados[(cluster, fecha)].append(turno)

    # Se actualizan los clusters asignados al instalador
    turnos_instaladores.setdefault((instalador, fecha), []).append((cluster, turno))

    # Se actualizan los instaladores asignados al cluster
    instaladores_cluster.setdefault((cluster, fecha), []).append((instalador, turno, direccion))

    return turno

# Se inicializan los diccionarios para llevar el registro de los turnos asignados y los clusters por instalador y por cluster
turnos_asignados = {}
turnos_instaladores = {}
instaladores_cluster = {}

# Se aplica la función para asignar turnos
df['Turno'] = df.apply(asignar_turno, axis=1, turnos_asignados=turnos_asignados, turnos_instaladores=turnos_instaladores, instaladores_cluster=instaladores_cluster)

# Guardar
ruta_archivo_salida = '/content/drive/MyDrive/Agrupados_asignados_instaladores.xlsx'
df.to_excel(ruta_archivo_salida, index=False)


### CREAR LAS ORDENES DE TRABAJO. En pdf se envia por correo la orden de trabajo indicando al instalador el domicilio, máquinas a instalar y el día de instalación

In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=c26699ef515b6710a97798bf7c6fd3f1ca7359b5b802ebc5a047b663502601ac
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [ ]:
from fpdf import FPDF
import os
import pandas as pd
# Cargar el archivo con los datos agrupados y asignados
agrupados = pd.read_excel("/content/drive/MyDrive/Agrupados_asignados_instaladores.xlsx", engine='openpyxl')
# Se crea el directorio para guardar los PDFs
output_dir = "/content/drive/MyDrive/PDF ORDENES DE TRABAJO"
os.makedirs(output_dir, exist_ok=True)
# Crear un PDF para cada 'Instalador Asignado'
for instalador in agrupados["Instalador Asignado"].unique():
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    # Se filtran datos por 'Instalador Asignado'
    subset = agrupados[agrupados["Instalador Asignado"] == instalador]
    # Agregar título al PDF
    pdf.cell(200, 10, txt=f"Orden de trabajo para el Instalador: {instalador}", ln=True, align='C')
    # Agrupación por 'FECHA COMP INICIAL' y 'Dirección'
    grouped = subset.groupby(['FECHA COMP INICIAL', 'DIRECCION', 'Latitud', 'Longitud', 'NUMERO'])

    for (fecha, direccion, latitud, longitud, numero), group in grouped:
        pdf.cell(200, 10, txt=f"FECHA COMP INICIAL: {fecha}", ln=True)
        pdf.cell(200, 10, txt=f"Dirección: {direccion} {numero}", ln=True)
                # Verificar si 'Solo_CP' es VERDADERO y agregar "Código Postal"
        if group["Solo_CP"].iloc[0]:  # Asumiendo que todos los valores en el grupo son iguales
            pdf.cell(200, 10, txt=f"Latitud: {latitud}, Longitud: {longitud} (Código Postal)", ln=True)
        else:
            pdf.cell(200, 10, txt=f"Latitud: {latitud}, Longitud: {longitud}", ln=True)

        for index, row in group.iterrows():
            pdf.cell(200, 10, txt=f"FAMILIA: {row['FAMILIA']}", ln=True)
            pdf.cell(200, 10, txt=f"BARRA: {row['BARRA']}", ln=True)
            pdf.cell(200, 10, txt=f"DESC. ARTICULO: {row['DESC. ARTICULO']}", ln=True)
            pdf.cell(200, 10, txt="------------------------------------------------------", ln=True)
    # Guardar el PDF
    pdf_filename = os.path.join(output_dir, f"{instalador}_asignado.pdf")
    pdf.output(pdf_filename)

print("Asignación completada y PDFs generados y guardados en '/content/drive/MyDrive/PDF ORDENES DE TRABAJO'.")


Asignación completada y PDFs generados y guardados en '/content/drive/MyDrive/PDF ORDENES DE TRABAJO'.


### ENVIO ORDENES DE TRABAJO POR EMAIL. Se envían todas las ordenes de trabajo a los instaladores asignados. (con mi email los prov10, 17, 18 y 30)

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os
import pandas as pd
import time

# Se configura del servidor SMTP y credenciales de la cuenta de correo
smtp_server = 'smtp.mail.yahoo.com'
smtp_port = 587
email_user = 'jdgf.2007@yahoo.es'
email_password = 'ydovcqwtubnwlmhc'

# Ruta del archivo Excel y la carpeta que contiene los archivos PDF
excel_path = '/content/drive/MyDrive/Capacidad_instaladores_dia_actualizado.xlsx'
folder_path = '/content/drive/MyDrive/PDF ORDENES DE TRABAJO'

# Leer el archivo Excel
df = pd.read_excel(excel_path, engine='openpyxl')

# Crear el objeto del mensaje
msg = MIMEMultipart()
msg['From'] = email_user
msg['Subject'] = 'Orden de trabajo instalación ECI'

# Cuerpo del mensaje
body = 'Adjunto encontrarás las ordenes de trabajo con la dirección del cliente y los equipos para su instalación.'
msg.attach(MIMEText(body, 'plain'))

# Función para enviar el correo con reintentos
def enviar_correo(email_to, msg):
    for i in range(3):  # Intentar 3 veces
        try:
            # Conectar al servidor SMTP
            server = smtplib.SMTP(smtp_server, smtp_port, timeout=10)
            server.starttls()
            server.login(email_user, email_password)
            text = msg.as_string()
            server.sendmail(email_user, email_to, text)
            server.quit()
            print(f"Correo enviado exitosamente a {email_to}")
            break
        except smtplib.SMTPServerDisconnected:
            print(f"Intento {i+1}: Conexión cerrada inesperadamente. Reintentando...")
            time.sleep(5)  # Esperar 5 segundos antes de reintentar
        except Exception as e:
            print(f"Error al enviar el correo a {email_to}: {e}")
            break

# Se adjunta a cada archivo PDF en la carpeta al correo electrónico
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        # Obtener el código del instalador desde el nombre del archivo PDF
        installer_code = filename.replace('_asignado.pdf', '')

        # Buscar el correo electrónico del instalador en el archivo Excel
        email_to = df.loc[df['Instaladores'] == installer_code, 'EMAIL'].values[0]

        # Asignar la dirección de correo del destinatario al mensaje
        msg['To'] = email_to

        file_path = os.path.join(folder_path, filename)
        attachment = open(file_path, 'rb')

        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename= {filename}')

        msg.attach(part)

        # Enviar el correo electrónico
        enviar_correo(email_to, msg)

print("Correos enviados")


Correo enviado exitosamente a jdgf.2007@gmail.com
Correo enviado exitosamente a jdgf.2007@gmail.com
Correo enviado exitosamente a jdgf.2007@gmail.com
Correos enviados


### SIGUIENTE DÍA DE GESTIÓN DE RESERVAS DE CLIENTES

###Se añaden al fichero que contiene todas las reservas con su evolución los datos de las reservas que ya han sido gestionadas, para disponer del dato agregado.

In [15]:
import pandas as pd

# Ruta de los archivos Excel
file_ultimas_reservas = '/content/drive/MyDrive/Ultimas reservas.xlsx'
file_agrupados_asignados = '/content/drive/MyDrive/Agrupados_asignados_instaladores.xlsx'

# Leer los archivos Excel
df_ultimas_reservas = pd.read_excel(file_ultimas_reservas, engine='openpyxl')
df_agrupados_asignados = pd.read_excel(file_agrupados_asignados, engine='openpyxl')

# Crear un nuevo DataFrame con las columnas adicionales
columnas_adicionales = ["Latitud", "Longitud", "Solo_CP", "Horas_instalacion", "cluster", "cluster_ajustado", "total_horas", "Instalador Asignado", "Latitud Instalador", "Longitud Instalador", "Turno"]
df_nuevo = df_ultimas_reservas.copy()

for columna in columnas_adicionales:
    df_nuevo[columna] = None

# Rellenar las columnas adicionales con los valores del fichero 'Agrupados_asignados_instaladores.xlsx' que coincidan con 'RESERVA' y 'BARRA'
for index, row in df_nuevo.iterrows():
    reserva = row['RESERVA']
    barra = row['BARRA']
    matching_row = df_agrupados_asignados[(df_agrupados_asignados['RESERVA'] == reserva) & (df_agrupados_asignados['BARRA'] == barra)]

    if not matching_row.empty:
        for columna in columnas_adicionales:
            df_nuevo.at[index, columna] = matching_row.iloc[0][columna]

# Guardar el nuevo DataFrame en un archivo Excel
output_file_instalaciones = '/content/drive/MyDrive/Ultimas_reservas_instalaciones.xlsx'
df_nuevo.to_excel(output_file_instalaciones, index=False)

print(f"El archivo '{output_file_instalaciones}' ha sido creado.")

El archivo '/content/drive/MyDrive/Ultimas_reservas_instalaciones.xlsx' ha sido creado.


### Se eliminan las reservas ya gestionadas.

In [16]:
import pandas as pd

# Ruta de los archivos Excel
file_ultimas_reservas = '/content/drive/MyDrive/Ultimas reservas.xlsx'
file_ultimas_reservas_instalaciones = '/content/drive/MyDrive/Ultimas_reservas_instalaciones.xlsx'

# Leer los archivos Excel
df_ultimas_reservas = pd.read_excel(file_ultimas_reservas, engine='openpyxl')
df_ultimas_reservas_instalaciones = pd.read_excel(file_ultimas_reservas_instalaciones, engine='openpyxl')

# Filtrar las filas del fichero 'Ultimas_reservas_instalaciones' que tienen algún dato en la columna 'Latitud'
df_ultimas_reservas_instalaciones_filtrado = df_ultimas_reservas_instalaciones[df_ultimas_reservas_instalaciones['Latitud'].notna()]

# Eliminar las filas del fichero 'Ultimas reservas' que ya se encuentran en el fichero 'Ultimas_reservas_instalaciones_filtrado' según las columnas 'RESERVA' y 'BARRA'
df_ultimas_reservas_reales = df_ultimas_reservas[~df_ultimas_reservas.set_index(['RESERVA', 'BARRA']).index.isin(df_ultimas_reservas_instalaciones_filtrado.set_index(['RESERVA', 'BARRA']).index)]

# Guardar el resultado en un nuevo archivo Excel
output_file = '/content/drive/MyDrive/Ultimas reservas_consolidado.xlsx'
df_ultimas_reservas_reales.to_excel(output_file, index=False)

print(f"El archivo '{output_file}' ha sido creado.")


El archivo '/content/drive/MyDrive/Ultimas reservas_consolidado.xlsx' ha sido creado.


### SE REPITEN LOS PASOS. FICHERO CON COORDENADAS Y HORAS DE INSTALACIÓN NECESARIAS. A las direcciones que no localiza se asignan las coordenadas del código postal.

In [17]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Se crea un objeto geolocalizador
geolocator = Nominatim(user_agent="mi_aplicacion", timeout=10)

# Archivo Excel
df = pd.read_excel('/content/drive/MyDrive/Ultimas reservas_consolidado.xlsx', sheet_name='Sheet1', engine='openpyxl')

# Función para obtener coordenadas con pausa
def obtener_coordenadas_con_pausa(row):
    direccion = f"{row['CALLE']}, {row['NUMERO']}, {row['CP']}, Madrid, España"
    try:
        location = geolocator.geocode(direccion)
        time.sleep(1)  # Pausa de 1 segundo entre solicitudes
        if location:
            return location.latitude, location.longitude, False
        else:
            # Intentar obtener coordenadas solo con el código postal
            direccion_cp = f"{row['CP']}, Madrid, España"
            location_cp = geolocator.geocode(direccion_cp)
            time.sleep(1)  # Pausa de 1 segundo entre solicitudes
            if location_cp:
                return location_cp.latitude, location_cp.longitude, True
            else:
                return None, None, False
    except GeocoderTimedOut:
        return obtener_coordenadas_con_pausa(row)

# Se aplica la función a cada fila del archivo
df['Latitud'], df['Longitud'], df['Solo_CP'] = zip(*df.apply(obtener_coordenadas_con_pausa, axis=1))

# Se añade la columna "Horas_instalacion" según las condiciones
def calcular_horas_instalacion(row):
    if row['FAMILIA'] == 810:
        return row['UNIDADES'] * 2
    elif row['FAMILIA'] == 811:
        return row['UNIDADES'] * 4
    elif row['FAMILIA'] == 812:
        return row['UNIDADES'] * 6
    elif row['FAMILIA'] == 813:
        return row['UNIDADES'] * 6
    elif row['FAMILIA'] == 816:
        return row['UNIDADES'] * 2
    elif row['FAMILIA'] == 818:
        return row['UNIDADES'] * 2
    elif row['FAMILIA'] == 820:
        return row['UNIDADES'] * 4
    elif row['FAMILIA'] == 822:
        return row['UNIDADES'] * 8
    elif row['FAMILIA'] == 824:
        return row['UNIDADES'] * 6
    else:
        return None
df['Horas_instalacion'] = df.apply(calcular_horas_instalacion, axis=1)

# Se guarda
output_path = '/content/drive/MyDrive/Ultimas reservas_coord_horas.xlsx'
df.to_excel(output_path, index=False)

# Formato de color rojo a las coordenadas asignadas solo por CP
wb = load_workbook(output_path)
ws = wb.active

# Estilo de color rojo para las celdas
red_fill = PatternFill(start_color="FFFF0000", end_color="FFFF0000", fill_type="solid")

# Se aplica el estilo a las celdas correspondientes
for index, row in df.iterrows():
    if row['Solo_CP']:
        ws[f'W{index + 2}'].fill = red_fill  # Columna C para Latitud
        ws[f'X{index + 2}'].fill = red_fill  # Columna D para Longitud

# Guardar los cambios
wb.save(output_path)

print("Las coordenadas y la nueva columna 'Horas_instalacion' se han añadido y el archivo se ha guardado como 'Ultimas reservas_coord_horas.xlsx'. Las coordenadas asignadas solo por CP se han marcado en color rojo.")


Las coordenadas y la nueva columna 'Horas_instalacion' se han añadido y el archivo se ha guardado como 'Ultimas reservas_coord_horas.xlsx'. Las coordenadas asignadas solo por CP se han marcado en color rojo.


### AGRUPAR POR CERCANIA A LOS CLIENTES. EPS = 0.02 rádio de 2,22K

In [18]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Cargar los datos
data = pd.read_excel('/content/drive/MyDrive/Ultimas reservas_coord_horas.xlsx', sheet_name='Sheet1')

# Función para agrupar por cercanía sin restricciones
def agrupar_por_cercania(data, eps=0.02, min_samples=1):
    agrupados = []

    # Se itera sobre cada FECHA COMP INICIAL
    for fecha in data["FECHA COMP INICIAL"].unique():
        subset = data[data["FECHA COMP INICIAL"] == fecha]

        # DBSCAN para agrupar por latitud y longitud
        coords = subset[["Latitud", "Longitud"]].values
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
        subset["cluster"] = db.labels_

        agrupados.append(subset)

    return pd.concat(agrupados)

# Guardar el resultado
agrupados = agrupar_por_cercania(data)
agrupados.to_excel("/content/drive/MyDrive/Clientes_agrupados_sin_restricciones.xlsx", index=False)

# Añadir columna "cluster_ajustado" según las reglas
def ajustar_clusters(data):
    data["cluster_ajustado"] = ""

    for cluster in data["cluster"].unique():
        cluster_data = data[data["cluster"] == cluster]

        total_hours = 0
        sub_cluster_id = 0
        lat_lon_dict = {}

        for index, row in cluster_data.iterrows():
            lat_lon = (row["Latitud"], row["Longitud"])

            if lat_lon in lat_lon_dict:
                # Sumar las horas de instalación para latitudes y longitudes iguales
                total_hours += row["Horas_instalacion"]
                data.loc[index, "cluster_ajustado"] = lat_lon_dict[lat_lon]
            else:
                # Se verifica si hay una sola longitud y latitud en el cluster
                if len(cluster_data["Latitud"].unique()) == 1 and len(cluster_data["Longitud"].unique()) == 1:
                    threshold = 8
                else:
                    threshold = 6

                # Se suman las horas de instalación de todas las filas con la misma latitud y longitud
                same_lat_lon_hours = cluster_data[(cluster_data["Latitud"] == row["Latitud"]) & (cluster_data["Longitud"] == row["Longitud"])]["Horas_instalacion"].sum()

                if total_hours + same_lat_lon_hours > threshold:
                    sub_cluster_id += 1
                    total_hours = row["Horas_instalacion"]
                else:
                    total_hours += row["Horas_instalacion"]

                cluster_ajustado = f"{cluster}_{sub_cluster_id}"
                data.loc[index, "cluster_ajustado"] = cluster_ajustado
                lat_lon_dict[lat_lon] = cluster_ajustado

    # Se suma el total de "Horas_instalacion" de cada "cluster_ajustado" por "FECHA COMP INICIAL" y asignar "manual" si es mayor de 8
    for fecha in data["FECHA COMP INICIAL"].unique():
        fecha_data = data[data["FECHA COMP INICIAL"] == fecha]
        for cluster_ajustado in fecha_data["cluster_ajustado"].unique():
            if fecha_data[fecha_data["cluster_ajustado"] == cluster_ajustado]["Horas_instalacion"].sum() > 8:
                data.loc[(data["FECHA COMP INICIAL"] == fecha) & (data["cluster_ajustado"] == cluster_ajustado), "cluster_ajustado"] = "MANUAL"

    return data

# Se ajustan los clusters y se guarda el resultado
data_ajustada = ajustar_clusters(agrupados)

# Crear una columna con la suma de horas por cluster y fecha
data_ajustada['total_horas'] = data_ajustada.groupby(['FECHA COMP INICIAL', 'cluster_ajustado'])['Horas_instalacion'].transform('sum')

# Función para aplicar el estilo
def highlight_equal_8(s):
    is_equal_8 = s == 8
    return ['background-color: yellow' if v else '' for v in is_equal_8]

# Se aplica el estilo
styled_data = data_ajustada.style.apply(highlight_equal_8, subset=['total_horas'])

# Guardar
styled_data.to_excel("/content/drive/MyDrive/Clientes_agrupados_n+1.xlsx", index=False)

print("Clusters ajustados y guardados en 'Clientes_agrupados_n+1.xlsx'")

<ipython-input-18-30ef7f31120b>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["cluster"] = db.labels_
<ipython-input-18-30ef7f31120b>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["cluster"] = db.labels_
<ipython-input-18-30ef7f31120b>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

Clusters ajustados y guardados en 'Clientes_agrupados_n+1.xlsx'


### ASIGNAR A LOS INSTALADORES POR CERCANIA Y EN FUNCIÓN DE SU CAPACIDAD DISPONIBLE (SI NO HAY INSTALADORES DISPONIBLES SE INDICA "SIN INSTALADOR")

In [19]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

agrupados = pd.read_excel("/content/drive/MyDrive/Clientes_agrupados_n+1.xlsx")
capacidad_instaladores_dia = pd.read_excel('/content/drive/MyDrive/Capacidad_instaladores_dia_actualizado.xlsx', engine='openpyxl')

# Extraer coordenadas de agrupados e instaladores
agrupados_coords = agrupados[["Latitud", "Longitud"]].values
instaladores_coords = capacidad_instaladores_dia[["Latitud", "Longitud"]].values
# Calcular la distancia entre cada grupo y cada instalador
distancias = cdist(agrupados_coords, instaladores_coords, metric="euclidean")

# Inicializar una columna para el instalador asignado
agrupados["Instalador Asignado"] = np.nan
agrupados["Latitud Instalador"] = np.nan
agrupados["Longitud Instalador"] = np.nan

# Inicializar un diccionario para contar las asignaciones por instalador y fecha
asignaciones_por_instalador = {}

# Iterar sobre cada grupo
for i in range(len(agrupados)):
    fecha = agrupados.loc[i, "FECHA COMP INICIAL"]
    horas_instalacion = agrupados.loc[i, "Horas_instalacion"]

    # Se filtran los instaladores que tienen horas disponibles para la fecha
    instaladores_disponibles = capacidad_instaladores_dia[capacidad_instaladores_dia[fecha] >= horas_instalacion]

    # Se obtienen las coordenadas de los instaladores disponibles
    disponibles_coords = instaladores_disponibles[["Latitud", "Longitud"]].values

    # Calcular la distancia entre el grupo actual y los instaladores disponibles
    distancias_disponibles = cdist([agrupados_coords[i]], disponibles_coords, metric="euclidean")

    # Se asigna el instalador más cercano disponible
    asignado = False
    while len(distancias_disponibles[0]) > 0:
        idx_min = distancias_disponibles.argmin()
        instalador_seleccionado = instaladores_disponibles.iloc[idx_min]
        horas_restantes = capacidad_instaladores_dia.loc[instalador_seleccionado.name, fecha] - horas_instalacion

        # Se verifica si el instalador ya tiene 2 asignaciones en la misma fecha
        if asignaciones_por_instalador.get((instalador_seleccionado["Instaladores"], fecha), 0) < 2:
            if horas_restantes >= 0:
                agrupados.loc[i, "Instalador Asignado"] = instalador_seleccionado["Instaladores"]
                agrupados.loc[i, "Latitud Instalador"] = instalador_seleccionado["Latitud"]
                agrupados.loc[i, "Longitud Instalador"] = instalador_seleccionado["Longitud"]

                # Restar las horas de instalación asignadas de las horas disponibles del instalador para esa fecha
                capacidad_instaladores_dia.loc[instalador_seleccionado.name, fecha] = horas_restantes
                # Actualizar el contador de asignaciones
                asignaciones_por_instalador[(instalador_seleccionado["Instaladores"], fecha)] = asignaciones_por_instalador.get((instalador_seleccionado["Instaladores"], fecha), 0) + 1
                asignado = True
                break
        else:
            # Eliminar el instalador no válido y recalcular distancias
            distancias_disponibles = np.delete(distancias_disponibles, idx_min, axis=1)
            instaladores_disponibles = instaladores_disponibles.drop(instalador_seleccionado.name)

    # Si no se asignó ningún instalador, marcar como "Sin instalador"
    if not asignado:
        agrupados.loc[i, "Instalador Asignado"] = "Sin instalador"

# Añadir la columna "Turno" al final del DataFrame
agrupados["Turno"] = np.nan
# Guardar el resultado en un nuevo archivo Excel
agrupados.to_excel("/content/drive/MyDrive/Agrupados_asignados_instaladores.xlsx", index=False)
capacidad_instaladores_dia.to_excel('/content/drive/MyDrive/Capacidad_instaladores_dia_actualizado.xlsx', index=False)


<ipython-input-19-68672c897c85>:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Prov12_inst1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  agrupados.loc[i, "Instalador Asignado"] = instalador_seleccionado["Instaladores"]


### SE ASIGNAN LOS TURNOS DE ENTREGA

In [ ]:
import pandas as pd

# Sobre el fichero anterior:
ruta_archivo = '/content/drive/MyDrive/Agrupados_asignados_instaladores.xlsx'
df = pd.read_excel(ruta_archivo, engine='openpyxl')

# Se define la función para asignar turnos
def asignar_turno(fila, turnos_asignados, turnos_instaladores, instaladores_cluster):
    cluster = fila['cluster_ajustado']
    fecha = fila['FECHA COMP INICIAL']
    horas = fila['Horas_instalacion']
    instalador = fila['Instalador Asignado']
    direccion = fila['DIRECCION']

    # Se verifica si el cluster y la fecha ya tienen turnos asignados
    if (cluster, fecha) in turnos_asignados:
        turnos_existentes = turnos_asignados[(cluster, fecha)]
    else:
        turnos_existentes = set()

    # Se verifica si el instalador ya tiene turnos asignados en la misma fecha
    if (instalador, fecha) in turnos_instaladores:
        clusters_instalador = turnos_instaladores[(instalador, fecha)]
    else:
        clusters_instalador = []

    # Se verifica si el cluster ya tiene instaladores asignados en la misma fecha
    if (cluster, fecha) in instaladores_cluster:
        lista_instaladores_cluster = instaladores_cluster[(cluster, fecha)]
    else:
        lista_instaladores_cluster = []

    # Criterios de asignación basado en las horas y los turnos
    if horas == 8:
        turno = 'DC'
    elif len(clusters_instalador) == 1:
        # Si el instalador solo tiene un cluster asignado en la misma fecha, asignar el turno contrario
        if clusters_instalador[0][1] == 'M':
            turno = 'T'
        else:
            turno = 'M'
    elif 'M' not in turnos_existentes and sum([h for h in turnos_asignados.get((cluster, fecha), []) if isinstance(h, int)]) < 7:
        turno = 'M'
    else:
        # Asignar turno de tarde 'T' solo si las horas no superan 4
        if horas <= 4:
            turno = 'T'
        else:
            turno = 'M'

    # Si hay dos o más instaladores en el mismo cluster y fecha con la misma dirección, asignar el mismo turno
    for inst in lista_instaladores_cluster:
        if inst[2] == direccion:
            turno = inst[1]
            break

    # Se verifica que un turno de tarde "T" no tenga más de 4 horas
    if turno == 'T' y horas > 4:
        turno = 'M'

    # Se verifica que un turno de mañana "M" no tenga más de 6 horas para el mismo instalador y día
    if turno == 'M' and sum([h for c, s in clusters_instalador if s == 'M' for h in turnos_asignados.get((c, fecha), []) if isinstance(h, int)]) + horas > 6:
        # Asignar turno de tarde 'T' solo si las horas no superan 4
        if horas <= 4:
            turno = 'T'
        else:
            turno = 'M'

    # Se actualizan los turnos
    turnos_asignados.setdefault((cluster, fecha), []).append(horas)
    turnos_asignados[(cluster, fecha)].append(turno)

    # Se actualizan los clusters asignados al instalador
    turnos_instaladores.setdefault((instalador, fecha), []).append((cluster, turno))

    # Se actualizan los instaladores asignados al cluster
    instaladores_cluster.setdefault((cluster, fecha), []).append((instalador, turno, direccion))

    return turno

# Se inicializan los diccionarios para llevar el registro de los turnos asignados y los clusters por instalador y por cluster
turnos_asignados = {}
turnos_instaladores = {}
instaladores_cluster = {}

# Se aplica la función para asignar turnos
df['Turno'] = df.apply(asignar_turno, axis=1, turnos_asignados=turnos_asignados, turnos_instaladores=turnos_instaladores, instaladores_cluster=instaladores_cluster)

# Guardar
ruta_archivo_salida = '/content/drive/MyDrive/Agrupados_asignados_instaladores.xlsx'
df.to_excel(ruta_archivo_salida, index=False)

###CREAR LAS ORDENES DE TRABAJO. En pdf se envia por correo la orden de trabajo indicando al instalador el domicilio, máquinas a instalar y el día de instalación

In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=2391965bde4fc0f2664dae80cefa63b97234b75fa470f1177c88b7a97d6231cb
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [ ]:
import pandas as pd
import numpy as np
from fpdf import FPDF
import os

# Cargar el archivo con los datos agrupados y asignados
agrupados = pd.read_excel("/content/drive/MyDrive/Agrupados_asignados_instaladores.xlsx", engine='openpyxl')

# Crear el directorio para guardar los PDFs si no existe
output_dir = "/content/drive/MyDrive/PDF ORDENES DE TRABAJO"
os.makedirs(output_dir, exist_ok=True)

# Generar un PDF para cada 'Instalador Asignado'
for instalador in agrupados["Instalador Asignado"].unique():
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Filtrar datos por 'Instalador Asignado'
    subset = agrupados[agrupados["Instalador Asignado"] == instalador]

    # Agregar título al PDF
    pdf.cell(200, 10, txt=f"Instalador Asignado: {instalador}", ln=True, align='C')

    # Agrupar por 'FECHA COMP INICIAL' y 'Dirección'
    grouped = subset.groupby(['FECHA COMP INICIAL', 'DIRECCION', 'Latitud', 'Longitud', 'NUMERO'])

    for (fecha, direccion, latitud, longitud, numero), group in grouped:
        pdf.cell(200, 10, txt=f"FECHA COMP INICIAL: {fecha}", ln=True)
        pdf.cell(200, 10, txt=f"Dirección: {direccion} {numero}", ln=True)
        pdf.cell(200, 10, txt=f"Latitud: {latitud}, Longitud: {longitud}", ln=True)

        for index, row in group.iterrows():
            pdf.cell(200, 10, txt=f"FAMILIA: {row['FAMILIA']}", ln=True)
            pdf.cell(200, 10, txt=f"BARRA: {row['BARRA']}", ln=True)
            pdf.cell(200, 10, txt=f"DESC. ARTICULO: {row['DESC. ARTICULO']}", ln=True)
            pdf.cell(200, 10, txt="------------------------------------------------------", ln=True)

    # Guardar el PDF en el directorio especificado
    pdf_filename = os.path.join(output_dir, f"{instalador}_asignado.pdf")
    pdf.output(pdf_filename)

print("Asignación completada y PDFs generados y guardados en '/content/drive/MyDrive/PDF ORDENES DE TRABAJO'.")

Asignación completada y PDFs generados y guardados en '/content/drive/MyDrive/PDF ORDENES DE TRABAJO'.


### ENVIO ORDENES DE TRABAJO POR EMAIL. Se envían todas las ordenes de trabajo a los instaladores asignados. (con mi email los prov10, 17, 18 y 30)

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os
import pandas as pd
import time

# Configuración del servidor SMTP y credenciales de la cuenta de correo
smtp_server = 'smtp.mail.yahoo.com'
smtp_port = 587
email_user = 'jdgf.2007@yahoo.es'
email_password = 'ydovcqwtubnwlmhc'

# Ruta del archivo Excel y la carpeta que contiene los archivos PDF
excel_path = '/content/drive/MyDrive/instaladores_coordenadas.xlsx'
folder_path = '/content/drive/MyDrive/PDF ORDENES DE TRABAJO'

# Leer el archivo Excel
df = pd.read_excel(excel_path, engine='openpyxl')

# Crear el objeto del mensaje
msg = MIMEMultipart()
msg['From'] = email_user
msg['Subject'] = 'Orden de trabajo instalación ECI'

# Cuerpo del mensaje
body = 'Adjunto encontrarás las ordenes de trabajo con la dirección del cliente y los equipos para su instalación.'
msg.attach(MIMEText(body, 'plain'))

# Función para enviar el correo con reintentos
def enviar_correo(email_to, msg):
    for i in range(3):  # Intentar 3 veces
        try:
            # Conectar al servidor SMTP
            server = smtplib.SMTP(smtp_server, smtp_port, timeout=10)
            server.starttls()
            server.login(email_user, email_password)
            text = msg.as_string()
            server.sendmail(email_user, email_to, text)
            server.quit()
            print(f"Correo enviado exitosamente a {email_to}")
            break
        except smtplib.SMTPServerDisconnected:
            print(f"Intento {i+1}: Conexión cerrada inesperadamente. Reintentando...")
            time.sleep(5)  # Esperar 5 segundos antes de reintentar
        except Exception as e:
            print(f"Error al enviar el correo a {email_to}: {e}")
            break

# Adjuntar cada archivo PDF en la carpeta al correo electrónico
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        # Obtener el código del instalador desde el nombre del archivo PDF
        installer_code = filename.replace('_asignado.pdf', '')

        # Buscar el correo electrónico del instalador en el archivo Excel
        email_to = df.loc[df['Instaladores por proveedor'] == installer_code, 'EMAIL'].values[0]

        # Asignar la dirección de correo del destinatario al mensaje
        msg['To'] = email_to

        file_path = os.path.join(folder_path, filename)
        attachment = open(file_path, 'rb')

        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename= {filename}')

        msg.attach(part)

        # Enviar el correo electrónico
        enviar_correo(email_to, msg)

print("Correos enviados")


Correo enviado exitosamente a prueba12@gmail.com
Correo enviado exitosamente a prueba5@gmail.com
Correo enviado exitosamente a prueba1@gmail.com
Correo enviado exitosamente a prueba22@gmail.com


KeyboardInterrupt: 

### Dato consolidado para Power BI

In [21]:
import pandas as pd

# Archivos
file_ultimas_reservas = '/content/drive/MyDrive/Ultimas_reservas_instalaciones.xlsx'
file_agrupados_asignados = '/content/drive/MyDrive/Agrupados_asignados_instaladores.xlsx'

# Leer los archivos Excel
df_ultimas_reservas = pd.read_excel(file_ultimas_reservas, engine='openpyxl')
df_agrupados_asignados = pd.read_excel(file_agrupados_asignados, engine='openpyxl')

# Columnas a completar
columnas_adicionales = ["Latitud", "Longitud", "Solo_CP", "Horas_instalacion", "cluster", "cluster_ajustado", "total_horas", "Instalador Asignado", "Latitud Instalador", "Longitud Instalador", "Turno"]

# Se rellenan las columnas adicionales con los valores del fichero 'Agrupados_asignados_instaladores.xlsx' que coincidan con 'RESERVA' y 'BARRA'
for index, row in df_ultimas_reservas.iterrows():
    reserva = row['RESERVA']
    barra = row['BARRA']
    matching_row = df_agrupados_asignados[(df_agrupados_asignados['RESERVA'] == reserva) & (df_agrupados_asignados['BARRA'] == barra)]

    if not matching_row.empty:
        for columna in columnas_adicionales:
            df_ultimas_reservas.at[index, columna] = matching_row.iloc[0][columna]

# Se guarda el nuevo archivo
output_file_instalaciones = '/content/drive/MyDrive/Ultimas_reservas_instalaciones_completadas.xlsx'
df_ultimas_reservas.to_excel(output_file_instalaciones, index=False)

print(f"El archivo '{output_file_instalaciones}' ha sido creado.")


<ipython-input-21-d8d8e3501c51>:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_ultimas_reservas.at[index, columna] = matching_row.iloc[0][columna]


El archivo '/content/drive/MyDrive/Ultimas_reservas_instalaciones_completadas.xlsx' ha sido creado.


### Para cambiar el formato del fichero de capacidad instaladores y poder usarlo en PowerBI

In [22]:
import pandas as pd

# Excel a transformar
file_path = '/content/drive/MyDrive/Capacidad_instaladores_dia_actualizado.xlsx'

# Archivo Excel
df_original = pd.read_excel(file_path, engine='openpyxl')

# Identificar las columnas de fechas
columnas_fechas = df_original.columns[5:]

# Transformar
df_transformado = df_original.melt(id_vars=['Empresa', 'Instaladores', 'Latitud', 'Longitud', 'EMAIL'],
                                   value_vars=columnas_fechas,
                                   var_name='FECHA',
                                   value_name='UNIDADES')

# Guardar el Excel modificado
output_file = '/content/drive/MyDrive/Capacidad_instaladores_dia_transformado.xlsx'
df_transformado.to_excel(output_file, index=False)

print(f"El archivo '{output_file}' ha sido creado.")


El archivo '/content/drive/MyDrive/Capacidad_instaladores_dia_transformado.xlsx' ha sido creado.
